# Assembly of Illumina reads

Answer the questions in the yellow blocks,
<div class="alert alert-block alert-warning">
**Exam question**
</div>
in a separate word document in case you need/want a course completion certificate. It is a requirement of ForBio, otherwise we can only hand out participation certificates, which might be sufficient for you to get credits, depending on your home university's policies. However there are not many questions and you can keep the answers to a minimum, and in any case we recommend you to at least think about the questions.

________



### De novo assembly
**1)** Connect to the cluster and then connect to the software environment.

In [ ]:
%%bash
module load Anaconda3/5.1.0
source activate forbio_env


________

**2)** Copy the folder for the assembly tutorial to your working directory and enter the folder:

In [ ]:
%%bash
cp -r /work/projects/forbio/tutorials_tobi/assembly /work/users/USERNAME/
cd assembly


_______

**3)** Prepare and run the de novo assembly job. This is what the basic syntax of a very simple abyss assembly command looks like (for more information see the [abyss manual](https://docs.google.com/document/d/1WUy2VYYWEfcPm3_PldXu8z_VVxCoQvfmIoL7ysucx-c/edit)): 
```
abyss-pe \
    --directory=OUTPUT_DIR \
    k=KMER_VALUE \
    name=SAMPLE_ID \
    in="/PATH/TO/MY_READS_R1.fastq /PATH/TO/MY_READS_R2.fastq"
```

Run this command in the command line (after inserting the correct values and file-paths) to test if you get it to work. If you get it to run successfully (no error messages), **STOP THE COMMAND** by pressing `Ctrl+D`. We are cancelling it for the reason that the command needs to run for a while (ca. 30 min) and we want to submit it as a jobscript to the queue-system with `sbatch` instead (instructions below).

<div class="alert alert-block alert-info">
INFO: You don't need to type the ` \`, which is only needed to continue your command over several lines in the command line. Instead you can just put everything in one line.
</div>

Knowing the abyss command syntax from above, **complete the abyss command** in the job-script `assembly_abyss.sh` in your tutorial folder. 

<div class="alert alert-block alert-info">
INFO: You only need to edit the fields containing "?" in the document.
</div>

You find the fastq files to assemble in the `de_novo_assembly` tutorial directory in the folder `fastq_reads`.

When the file is set up, submit the job using `sbatch`. Check if the job is running with `squeue -u YOUR_USERNAME`.

<div class="alert alert-block alert-info">
INFO: The job will run in the background for ca. 30 min. In the meantime move on to the next step.
</div>

Below again the commands to edit, submit and check your script:

In [ ]:
%%bash
# do the changes in the file by opening it in the nano text editor
nano assembly_abyss.sh
# submit the job
sbatch assembly_abyss.sh
# check if the job is running
squeue -u YOUR_ABEL_USER_ID


_____

**4)** Try out different kmer values. Change the kmer value in the job script and submit again. Try at least 3 different values between 20 and 80.

<div class="alert alert-block alert-success">
If all assemblies (different kmer values) are running properly, you have ca. 30 min of time until all your assemblies are finished. Go grab a coffee and/or seize the time to have a look at some of the publications in the [course literature folder](https://drive.google.com/drive/u/0/folders/1VjMMVMf3XWi_Vm7-r6C6_GXuFwSzpk_z).

I recommend you to have a look at [this article](https://peerj.com/articles/5175/), not only because of shameless self-advertizing, but because of the description of the workflow, which is very similar to this exercise. 


</div>



______

**5)** Evaluate assembly results. There are a lot of files in the assembly output. The file containing the final contig sequences is stored in the assembly output folder under the name `T_pella5-contigs.fa`. Check it out using the following commands:

In [ ]:
%%bash
# print first lines of file to screen
head -n 10 T_pella5-contigs.fa
# print last lines of file to screen
tail -n 10 T_pella5-contigs.fa


Try to figure out what information is stored in the sequence headers.

Have a look at the contig sequence length distribution. I wrote a small program (`plot_contig_length.sh`) that plots the distribution of sequence lengths to the screen. The program is found in the `scripts` folder and can be executed like this (the script will take a minute or so to finish):


In [ ]:
%%bash
sh scripts/plot_contig_length.sh ./path/to/T_pella5-contigs.fa 

<div class="alert alert-block alert-warning">
**Question:** **How many contigs** were produced? And what about the **length distribution of the contig sequences**? What differences can you find between the different abyss runs with different kmer values? Decide which kmer value you think produced the best de novo assembly and explain why you think so. Continue with your chosen contig file for the following steps.
</div>


_____________________

### Blast contig against NCBI GeneBank:

**1)** Extract the longest contig from the contig file using the program `fp.py`, which you can install with the command below:

In [ ]:
%%bash
wget https://raw.githubusercontent.com/mtop/ngs/master/fp.py

You execute this program like this:

In [ ]:
%%bash
python2.7 /path/to/fp.py -h

Use `fp.py` to extract the longest contig sequence from the contig file. Extract it and write it to a separate fasta file `longest_contig.fasta`:

In [ ]:
%%bash
python2.7 /path/to/fp.py --longest ./path/to/contigs/T_pella5-contigs.fa > target_contigs/longest_contig.fasta


______

**2)** Then run a blast search against NCBI GeneBank to see what this sequence represents. You can use the blast commandline tool to run a blast search against the online database:

In [ ]:
%%bash
module load blast+
blastx -query target_contigs/longest_contig.fasta -db nr -out target_contigs/blast_results.txt -remote

I prepared a job-script, which you find at `scripts/run_blast_search.sh`. Execute the script from the folder `target_contigs/` as it needs to find the file `longest_contig.fasta`. The search will take at least 10 minutes.


______

### Blast contigs against mitochondrial genome:
Let's say you don't want to search through all of GeneBank but instead want to test if you have contigs representing a certain sequence, say the mitochondrial genome. In that case we can find a mitochondrial genome sequence of a close ancestor on GeneBank, download it and build our own blast reference with it.

**1)** Since we are using hummingbird data, let's see if we can find any contigs of mitochondrial origin by blasting the contigs against the mitochondrial genome of a related hummingbird. I stored a reference sequence of the hummingbird *Amazilla versicolor* in the subfolder `reference_sequences`. You first have to create a blast database with that sequence using the following command:

In [ ]:
%%bash
makeblastdb -dbtype nucl -in reference_sequences/amazilla_versicolor_mitochondrion.fasta

**2)** Now you can run a blast search against this local database using all contigs that were assembled:

In [ ]:
%%bash
blastn -query ./path/to/T_pella5-contigs.fa -db reference_sequences/amazilla_versicolor_mitochondrion.fasta -out target_contigs/blast_mitoch_results.txt -outfmt 6

Inspect the output file, which contains a list of all contigs that could be matched to the mitochondrial genome. The field-codes of the columns in the output file:
`query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score`



______

### Extract contigs matching bait set:

**1) Generate UCE fasta library.** The data we are working with were generated using an Ultraconserved Elements probe kit, which **targets 2560 UCE loci** (one probe per target locus). Let's now see how many and which contigs match with the UCE probe sequences. Create a local blast database for the file `reference_sequences/Tetrapods-UCE-2.5Kv1.fasta` (see above).

_______

**2) Run blast search.** Now blast the contigs file against the UCE reference database, using the command from above and storing the output in the folder `target_contigs` with the file named `blast_uce_results.txt`.

______

**3) Evaluate results.** Look at the output file from the blast command.

<div class="alert alert-block alert-warning">
**Question:** What can you say about the matches that were found? How similar are they to the references in the reference library (= probe sequences)?
</div>

The command below returns the number of contigs matching the reference library.

In [ ]:
%%bash
cat target_contigs/blast_uce_results.txt | wc -l

<div class="alert alert-block alert-warning">
**Question:** Do you have any comments on the number of matches, regarding the number of loci we targeted? If it strikes you as weird, can you think of any explanation?
</div>


_____

**4) Get sequences of contigs matching baits.** The blast output from the command above tells us which contig headers represents matches to the UCE bait set. Now we want to extract those sequences from the contig FASTA file and export them to a new FASTA file only containing target sequences. For that we can generate a loop, which iterates through every line of the blast output, gets the sequence ID and searches for a match in the contig file, which it prints to a new file `uce_contig_sequences.fasta` (insert the appropiate paths into the commands):

In [ ]:
%%bash
for i in $(cat target_contigs/blast_uce_results.txt | cut -f 1 | sort -u); do grep ">$i " -A 1 ./path/to/T_pella5-contigs.fa ; done > target_contigs/uce_contig_sequences.fasta

Since the command takes a few minutes it is best to interrupt it with `ctrl+c` and **RUN IT USING A JOB SCRIPT**. I prepared a script `extract_uce_contigs.sh` in the `scripts` folder, which you need to edit (`nano`) by inserting the correct paths to the input files (see comments in script for instructions). Once the paths are correct, submit the script with:

In [ ]:
%%bash
sbatch scripts/extract_uce_contigs.sh

<div class="alert alert-block alert-info">
If you are eager to fully understand the bash loop, feel free to ask, but it's not essential to understand the complete syntax for this purpose. We will later see that there are handy pipelines that do this step for us, so we don't need to mess around with bash loops, etc.
</div>


_______

**5) Check out sequence length distribution:** Let's see how long our UCE contigs are, using the plotting script in the `scripts` folder:

In [ ]:
%%bash
sh scripts/plot_contig_length.sh target_contigs/uce_contig_sequences.fasta

<div class="alert alert-block alert-warning">
**Question:** Would you say that this target capture experiment worked well? Evaluate in terms of percent of targeted loci that were recovered with a sufficient length (say > 450bp). As side information, the length of the bait sequences that were used was 120bp (using only one bait per locus, as mentioned above).
</div>



________

### Reference-based assembly

Now we want to use these extracted contig sequences as references to map the raw reads against in a step referred to as reference based assembly. For this purpose we use the `bwa` mapper.


_______________

**1)** Index this subsample in order ot use it as a reference

In [ ]:
%%bash
bwa index target_contigs/uce_contig_sequences.fasta


____________

**2)** Run the reference assembly with the command below. You can customize this command a lot, since bwa offers a lot of different flags you can play with to fine-tune your read mapping. For an explanation of all available flags, check out the [bwa tutorial](http://bio-bwa.sourceforge.net/bwa.shtml#3). Here is an example of some of the flags (including example values), which are commonly used: `-k 50 -w 21 -d 100 -r 1.5 -c 10000 -A 1 -B 4 -O 10 -E 5 -L 4 -U 2 -M`

In [ ]:
%%bash
#bwa mem -P target_contigs/uce_contig_sequences_subsample.fasta fastq_reads/T_pella5_R1.fastq fastq_reads/T_pella5_R2.fastq > reference_assembly/reference_assembly.sam
bwa mem -M target_contigs/uce_contig_sequences.fasta fastq_reads/T_pella5_R1.fastq fastq_reads/T_pella5_R2.fastq > reference_assembly/reference_assembly.sam

If this command takes too long, use `ctrl+c` to cancel it and run it using a job script. I prepared a script, which you can find under `scripts/reference_assembly.sh`

___________

**3)** The output will be in SAM format, but we need to convert it into BAM format (the binary version).

In [ ]:
%%bash
samtools view -b -o reference_assembly/reference_assembly.bam -S reference_assembly/reference_assembly.sam


_____________

**4)** Next you need to sort the reads inside the BAM file.

In [ ]:
%%bash
samtools sort reference_assembly/reference_assembly.bam reference_assembly/reference_assembly_sorted


_______________

**5)** Finally, index the BAM file so we can view it in the next step:

In [ ]:
%%bash
samtools index reference_assembly/reference_assembly_sorted.bam 


_____

**6)** View the BAM file using `samtools tview`:

In [ ]:
%%bash
samtools tview reference_assembly/reference_assembly_sorted.bam


<div class="alert alert-block alert-info">
For an even nicer displaying of the BAM files, you can use the software [Tablet](https://ics.hutton.ac.uk/tablet/download-tablet/). In order to do this you need to download Tablet to your computer, copy the sorted BAM file (`reference_assembly_sorted.bam`) and its' index file (`reference_assembly_sorted.bam.bai`) to your computer as well, open Tablet and drag and drop the two files into the open Tablet window.
</div>

_________

**7)** View the reads in relation to the reference by adding the path to the reference FASTA file to the end of the command

In [ ]:
%%bash
samtools tview reference_assembly/reference_assembly_sorted.bam target_contigs/uce_contig_sequences.fasta


_____

**8)** Check out the reference assemblies for different loci. Have a look at the reference FASTA file to identify some loci you want to display (look at the contig ID in the FASTA header).

In [8]:
%%bash
head -n 10 target_contigs/uce_contig_sequences.fasta

>100103 164 310
ATGAAATGTTGCCTTTATTTATGTTCATGTGAAATTCTGAGGAGGAGCTCTTACACTTAAATGAACCTGCAAGCAGCCCATAGAAAAGCAAGTGCCTGCAGCGCTAATAATGAGAACTGACAAGGCGATCCTGCTGTGTTTATTTATGACCTTCTGCCTTTGTT
>100121 665 3824
ATAGGTCATTCTTTCATTATACTATTGTTAAATTACAGACTACTACAAAAGGACATGCTATCTTGAATAAGAGAAAGGGCAGGGTTGGGTAGGGACAATCAGGATAACAGAAGAGTGCACTTAATTTGAGGCCTGCATATGAGCACTGCAGTGATCATTTAGTTCCAGCGGAATGAAATATGCCTTGGGCGATCTCCTCCTTCCTTCACCAGGGGCTTGTGGAATTCCCTGCCAGCACGTGAATGGATAGCAGCCCAGCAATATTCACAGTAATACTGCAGACAGGTAACATTAGCACAGAAAAACGGAGCAAATTTTCCACCACAACGGGCCCCCTGACATTCATCACATAGCTGATCATCCAAGACATATGGCTTAACTTCCACCTGTATGCAGAAAATGAGAAGAAAGCTTACAATTCTTCTTAACAATGGAGGTAAGAAAAAAACCCATACTATTTGCTACCTGTGTTCTAGCAGGACTAGGATCACAATTCTGTAGTAGGGAACACTACCCTCAAGGATGCTTCCCCTCATTACACTCAATCATAGCCATTAAAAACTCAAAAGTATAGGCACTTAGATTAAAAACAACCCTGCTCACACTCTCCTCCTTGATCCGTAAAAAAACCCAGAAACCGATGCTGACCATCAGTATCACACCTC
>100138 900 21907
AATGCCAAAATAAAATTACAAAAATAATACTGCTTTGAAAGTTTTCTGCTTGCTTTTAGCTGACAGGTCTCTAAGCGGATATGGCTTTTTCCACCTGCAATCAAATGTAATAAACCTT

Now add the contig ID you want to view to the `samtools tview` command using the `-p` flag.

In [ ]:
%%bash
samtools tview -p 100121 reference_assembly/reference_assembly_sorted.bam

<div class="alert alert-block alert-warning">
**Question:** Do you notice something wrong with the assemblies? Think about what we talked about earlier in the course regarding duplicate reads.
</div>

The next command removes duplicate reads from the reference assembly:

In [ ]:
%%bash
module load picard-tools
java -jar $PICARD_JAR MarkDuplicates I=reference_assembly/reference_assembly_sorted.bam O=reference_assembly/reference_assembly_sorted_no_duplicates.bam M=reference_assembly/picard_out.txt REMOVE_DUPLICATES=true VALIDATION_STRINGENCY=LENIENT

Inspect the resulting BAM file. You will have to index the new BAM file first, using `samtools index my_dupl_free_bam_file.bam` Compare the same locus before and after removing duplicate reads using `samtools tview`.


<div class="alert alert-block alert-warning">
**Question:** Insert a screenshot from before and after duplicate removal of the same locus into your report.
</div>


<div class="alert alert-block alert-success">
**Congrats!!** You made it to the end of this tutorial, which means you now know the necessary tools to run a de novo assembly, to extract your target contigs, to run a reference based assembly and to view and evaluate your results!

</div>